In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [56]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import re

import joblib
import sklearn.preprocessing
import sklearn.neighbors


from util import get_data
from util import mapping
from util import clean_data
from util import config
from util import plot_clusters
from util import process_data

In [53]:
trips = clean_data.load_clean_ridewgps_trips()
trips.to_csv(config.CLEAN_DATA_PATH + 'trips.csv')
trips.head()

,id,departed_at,duration,distance,elevation_gain,description,name,avg_speed,max_speed,moving_time,...,update_days,if_updated,elevation_net,elevation_total,avg_slope,photos,big_user,crow_distance,if_weekend,prop_moving
0,23209396,2018-04-30T02:50:43-07:00,0.619444,2.659848,123.437454,,04/30/18,5.002846,7.048852,0.531667,...,0,False,17.273481,229.601427,1.634872,False,True,0.940007,False,0.858296
1,54372107,2020-08-15T03:04:55-07:00,0.540556,2.762263,142.176885,,08/15/20,5.171163,5.960829,0.534167,...,0,False,-30.236712,314.590483,2.156983,False,True,0.012711,False,0.988181
2,48496079,2020-05-08T12:11:50-07:00,0.535278,2.777424,480.997881,,05/08/20,5.518063,11.760722,0.503333,...,0,False,87.563328,874.432434,5.962798,False,True,0.222277,False,0.940322
3,40433541,2019-09-23T11:49:34-07:00,0.580000,2.779512,57.147474,,09/23/19,5.192650,5.432662,0.535278,...,0,False,15.291757,99.003191,0.674601,False,True,0.002539,False,0.922893
4,47936294,2020-04-28T15:55:10-07:00,1.990278,2.795935,299.426029,,04/28/20,5.470311,7.853530,0.511111,...,0,False,-7.594531,606.446588,4.108011,False,True,0.005981,False,0.256804


In [61]:
useful_cols = ['distance', 'avg_slope', 'avg_speed', 'prop_moving']
rides_use = trips[useful_cols].copy().set_index(trips['id'])
num_cols = [col for col in useful_cols if rides_use[col].dtype in ['float64']]

col_importance = [1, 2, 1, 1]
rides_use = process_data.scale_dataset(rides_use[num_cols], col_importance)


In [67]:
a = pd.read_csv(config.MODEL_PATH + 'feature_scaling.csv', index_col=0)
a.loc['mean', 'distance']

30.414226132704712

In [54]:
rides_use.head()
rides_use.to_csv(config.PROCESSED_DATA_PATH + 'trips.csv')

In [71]:
presets, labs = process_data.set_presets()
process_data.apply_scaling(presets)

,distance,avg_slope,avg_speed,prop_moving
0,-0.995951,-0.735381,-1.130016,-1.035077
1,-0.752016,4.477657,-1.747298,-1.035077
2,0.711598,1.002298,0.413188,-0.346096
3,-0.508080,-0.735381,1.339110,1.031866
4,-0.995951,9.690695,-1.747298,-1.035077
5,2.663082,-0.735381,0.413188,-0.346096


In [98]:
df, labs = process_data.set_presets()
process_data.apply_scaling(presets)
# scaler = pd.read_csv(config.MODEL_PATH + 'feature_scaling.csv', index_col=0)
# for col in scaler.columns:
#     df[col] = ((df[col] - scaler.loc['mean', col]) / scaler.loc['std', col]
#                    * scaler.loc['column_importance', col])

,distance,avg_slope,avg_speed,prop_moving
0,-1.532412,-5.488595,-4.565193,-12.989429
1,-1.520511,3.569992,-4.755711,-12.989429
2,-1.449105,-2.469066,-4.088897,-8.242480
3,-1.508610,-5.488595,-3.803120,1.251419
4,-1.532412,12.628580,-4.755711,-12.989429
5,-1.353898,-5.488595,-4.088897,-8.242480


In [112]:
chosen = pd.DataFrame(np.array([[10, 5, 10, 0.8]]), columns=presets.columns)
process_data.apply_scaling(chosen)

,distance,avg_slope,avg_speed,prop_moving
0,-0.995951,4.477657,-1.130016,-0.346096


In [128]:
df.head()

,distance,avg_slope,avg_speed,prop_moving
id,,,,
23209396,-1.354056,-1.369856,-2.672341,0.055552
54372107,-1.349059,-0.462595,-2.620392,0.950435
48496079,-1.348320,6.150691,-2.513324,0.620694
40433541,-1.348218,-3.038500,-2.613760,0.500612
47936294,-1.347417,2.927666,-2.528063,-4.088614


In [131]:
df = pd.read_csv(config.PROCESSED_DATA_PATH + 'trips.csv', index_col=0)
chosen = df.iloc[[10]]
feature_importance = [1, 1, 1, 1]
tree = sklearn.neighbors.KDTree(df * feature_importance, leaf_size=20)

dist, ind = tree.query(chosen, k=5)
ind

array([[  10,  353,  498, 1027, 1087]])